# Chi-Square Goodness of Fit Test

---

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.power import TTestIndPower

plt.style.use("fivethirtyeight")
%matplotlib inline

---

Quando si opera con variabili *categoriche a più di due livelli*, una delle analisi più frequenti è la **goodness of fit**. Si cerca di valutare se la distribuzione delle osservazioni all'interno della variabile categorica segue i valori *attesi* o meno. In altre parole, la *statistic* di riferimento in contesti simili è la *count*. Il test di riferimento in questi scenari è il **chi-square GOF**, e come gli altri presenta delle condizioni di validità:

- **Independence**: le osservazioni raccolte nel sample devono essere indipendenti, quindi dev'essere applicato il *random sampling/assignment* e, nel caso di *sampling without replacement*, il *sample size* dev'essere inferiore al 10% della popolazione. Inoltre, ogni istanza non può appartenere a più di un livello della variabile categorica, cioè può contribuire al *count* totale solo per una di esse.


- **Sample Size**: ogni livello della variabile categorica deve avere almeno 5 *expected cases*.

Per lavorare con i *counts* ci serve una nuova *statistic*, la **chi-square statistic**, che ci aiuta a misurare quanto gli *observed counts* differiscano dagli *expected counts*. Si calcola così.

$$\large \chi^2=\sum_{i=1}^{k}\frac{(O-E)^2}{E}$$

In altre parole, la *chi-square* si calcola come la somma, per tutti i livelli della variabile categorica, del rapporto fra la differenza di *observed* ed *expected* al quadrato normalizzata per gli *expected*.

La corrispondente distribuzione, la **chi-square distribution**, ha come solo parametro i *degrees of freedom*, che ne influenzano il centro, l'andamento e lo spread. Il valore di $df$ va fissato al numero di livelli della variabile categorica meno uno.

$$\large df=k-1$$

E' possibile anche isolare una certa categoria o un insieme di categorie per incentrare il test solo su di esse: basta raggruppare diversamente i dati e modificare opportunamente il *significance level*, seguendo una sorta di *Bonferroni Correction*.

---

Il **chi-square test** può essere utilizzato anche per verificare se dei dati seguono una certa distribuzione parametrica, come la binomiale, la Poisson o la normale.

Ad esempio, supponiamo che una persona affermi che gli esiti dei tiri liberi di Larry Bird seguano un andamento binomiale con $p=0.8$. Guardiamo i dati che abbiamo a disposizione, per un totale di 388 coppie di tiri liberi:

- Zero canestri: 5
- Un canestro: 82
- Due canestri: 251

Calcoliamo gli **expected counts**:

In [2]:
total_attempts = 338

expected_zero = stats.binom.pmf(k = 0, n = 2, p = 0.8) * total_attempts
expected_one = stats.binom.pmf(k = 1, n = 2, p = 0.8) * total_attempts
expected_two = stats.binom.pmf(k = 2, n = 2, p = 0.8) * total_attempts

expected_zero, expected_one, expected_two

(13.519999999999996, 108.16, 216.32)

In [3]:
observed = [5, 82, 251]
expected = [expected_zero, expected_one, expected_two]

chisq_statistic, p_value = stats.chisquare(
    f_obs = observed,
    f_exp = expected)

chisq_statistic, p_value

(17.256102071005916, 0.00017901319576144793)

Possiamo rigettare l'ipotesi nulla, ma non sapppiamo ancora se il problema è nella distribuzione binomiale o nel suo parametro $p$.

Modifichiamo l'esempio: l'ipotesi nulla ora è che il numero di successi di Bird su due tiri liberi segua un andamento binomiale.

Usiamo i dati per calcolare $\hat{p}$, una stima del valore reale $p$, cioè la probabilità di fare canestro da tiro libero.

In [4]:
# Al denominatore moltiplichiamo per due
# Perché nei 338 tentativi ha tirato sempre due volte

phat = (5*0 + 82*1 + 251*2)/((5 + 82 + 251) * 2)
phat

0.863905325443787

In [5]:
total_attempts = 338

expected_zero = stats.binom.pmf(k = 0, n = 2, p = phat) * total_attempts
expected_one = stats.binom.pmf(k = 1, n = 2, p = phat) * total_attempts
expected_two = stats.binom.pmf(k = 2, n = 2, p = phat) * total_attempts

expected_zero, expected_one, expected_two

(6.260355029585801, 79.47928994082842, 252.26035502958575)

In [6]:
observed = [5, 82, 251]
expected = [expected_zero, expected_one, expected_two]

# Forziamo un df in meno perché l'abbiamo perso
# Dato che abbiamo stimato phat dai dati!

chisq_statistic, p_value = stats.chisquare(
    f_obs = observed,
    f_exp = expected,
    ddof = len(observed)- 1 - 1)

chisq_statistic, p_value

(0.3399809101747712, 0.5598402411323238)

Non ci sono prove contro l'ipotesi nulla!

---

**[Esempio]** Voglio verificare se c'è stata discriminazione nella scelta dei cittadini che devono far parte di una giuria in tribunale. So che la distribuzione di etnicità nell'intera popolazione è la seguente:

In [7]:
white = 0.8029
black = 0.1206
nat_am = 0.0079
asian = 0.0292
other = 0.0394

Nell'anno passato sono state scelte 2500 persone per far parte di una giuria. Con questo dato posso calcolare gli *expected counts*.

In [8]:
n = 2500

exp_white = int(n * white)
exp_black = int(n * black)
exp_nat_am = int(n * nat_am)
exp_asian = int(n * asian)
exp_other = int(n * other) + 2
# correggo other così che la somma degli expected sia 2500

exp_white + exp_black + exp_nat_am + exp_asian + exp_other

2500

Ho a disposizione anche gli *observed count*:

In [9]:
obs_white = 1920
obs_black = 347
obs_nat_am = 19
obs_asian = 84
obs_other = 130

Applichiamo il test:

In [10]:
observed = np.array([obs_white, obs_black, obs_nat_am, obs_asian, obs_other])
expected = np.array([exp_white, exp_black, exp_nat_am, exp_asian, exp_other])

In [11]:
chisq_statistic, p_value = stats.chisquare(
    f_obs = observed,
    f_exp = expected)

chisq_statistic, p_value

(21.45873502723175, 0.000256784904028504)

C'è discriminazione!

---